In [2]:
import csv
csv_counter = 0
with open("checkins.dat") as dat_file:
    with open("checkins.csv", "w") as csv_file:
        csv_writer = csv.writer(csv_file)
        for dat_line in dat_file:
            new_dat_line = map(str.strip, dat_line.split('|'))
            if len(new_dat_line) == 6 and new_dat_line[3] and new_dat_line[4]:
                csv_writer.writerow(new_dat_line)
                csv_counter += 1
    

In [3]:
import pandas as pd
df = pd.read_csv("checkins.csv")

In [4]:
df.head(10)

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18
5,984483,1030290,955969,32.221743,-110.926479,2012-04-21 17:58:54
6,984685,304253,23558,40.650000,-73.950000,2012-04-21 18:19:34
7,984470,720850,749715,33.448377,-112.074037,2012-04-21 17:02:47
8,984610,1639666,442605,33.414768,-111.909309,2012-04-21 18:04:58
9,984653,1647192,23558,42.358431,-71.059773,2012-04-21 18:23:22


In [5]:
import numpy as np
X = np.vstack((np.array(df['latitude']), np.array(df['longitude'])))

In [6]:
X = X.T

In [22]:
X[:100000, :].shape

(100000L, 2L)

In [7]:
from sklearn.cluster import MeanShift
clustering = MeanShift(bandwidth = 0.1)
clustering.fit(X[:100000, :])

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=None, seeds=None)

In [23]:
clustering.cluster_centers_

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ...,
       [ -37.8229826 ,  145.1811902 ],
       [ -41.2924945 ,  174.7732353 ],
       [ -45.0311622 ,  168.6626435 ]])

In [10]:
clustering.labels_

array([ 5,  7, 30, ..., 25, 19,  4], dtype=int64)

In [38]:
counter = []
for i in range(clustering.cluster_centers_.shape[0]):
    if ((clustering.labels_ == i).sum() > 15):
        counter.append(i)

int

In [43]:
f = []
for i in counter:
    f.append(clustering.cluster_centers_[i])
len(f)

592

In [48]:
big_centers = np.array(f)

In [51]:
big_centers

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ...,
       [  41.61853175,  -88.44556818],
       [  39.2494686 ,  -77.1821271 ],
       [  38.65877915,  -76.8856871 ]])

In [54]:
offices = np.array([[33.751277, -118.188740], [25.867736, -80.324116], [51.503016, -0.075479], [52.378894, 4.885084],
                   [39.366487, 117.036146], [-33.868457, 151.205134]])

In [64]:
len(offices)

6

In [66]:
distances = []
from scipy.spatial.distance import euclidean
for i in range(len(f)):
    for j in range(len(offices)):
        distances.append(euclidean(big_centers[i], offices[j]))

In [74]:
distances.index(min(distances))%6

5

In [75]:
euclidean(big_centers[distances.index(min(distances))/6], offices[distances.index(min(distances))%6])

0.007834758163107856

In [76]:
big_centers[distances.index(min(distances))/6]

array([-33.86063043, 151.20477593])

In [79]:
with open("clustering_cruise_answer.txt", "w") as fout:
    fout.write(' '.join(map(str, big_centers[distances.index(min(distances))/6])))